### 导包和变量设置

In [1]:
# 引入PyPDFDirectoryLoader，可以从文件夹中一次性加载所有pdf文件
# 然后使用RecursiveCharacterTextSplitter对解析出来的文档进行切分，主要根据分隔符，chunk_size以及overlap等

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers.bm25 import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import os
import gc


DOCS_DIR = '/root/autodl-tmp/dataset/rag/A_document'
EMB_MODEL = '/root/autodl-tmp/models/bge-large-zh-v1_5'
RERANK_MODEL = "/root/autodl-tmp/models/bge-reranker-large"
PERSIST_DIR = '/root/autodl-tmp/vectorDatabase/faiss_llmsherpa'
QUERY_DIR = '/root/autodl-tmp/dataset/rag/A_question.csv'
SUB_DIR = '/root/autodl-tmp/dataset/rag/submit.csv'
query = pd.read_csv(QUERY_DIR)
sub = pd.read_csv("/root/autodl-tmp/dataset/rag/submit_example.csv")
display(query.head(3))
display(sub.head(3))

/root/miniconda3/envs/dl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,ques_id,question
0,1,根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？
1,2,告诉我2022年联通产业互联网收入的同比增长速度。
2,3,根据2022年度报告，中国联通的企业定位是什么？


,ques_id,question,answer,embedding
0,1,根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？,我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字...,"-0.02707982249557972,-0.009818901307880878,-0...."
1,2,告诉我2022年联通产业互联网收入的同比增长速度。,我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字...,"-0.02707982249557972,-0.009818901307880878,-0...."
2,3,根据2022年度报告，中国联通的企业定位是什么？,我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字...,"-0.02707982249557972,-0.009818901307880878,-0...."


### PDF文档解析和切分

In [ ]:
# # # 进行数据加载
# # loader = PyPDFDirectoryLoader(DOCS_DIR)

# # 使用 PyPDFDirectoryLoader 加载所有 PDF 文件
# pdf_loader = PyPDFDirectoryLoader(DOCS_DIR)
# documents = pdf_loader.load()

# # 使用 LLMSherpaFileLoader 加载文档
# sherpa_loader = LLMSherpaFileLoader(
#     new_indent_parser=True,
#     apply_ocr=False,
#     strategy="text",
#     # llmsherpa_api_url="http://127.0.0.1:5001/api/parseDocument?renderFormat=all&useNewIndentParser=true&applyOcr=yes"
#     llmsherpa_api_url="http://0.0.0.0:5001/api/parseDocument?renderFormat=all",
# )

# loaded_documents = [sherpa_loader.load(doc) for doc in documents]

# from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader

# loader = LLMSherpaFileLoader(
#     file_path="/root/autodl-tmp/dataset/rag/A_small/AF01.pdf",
#     new_indent_parser=True,
#     apply_ocr=False,
#     strategy="text",
#     llmsherpa_api_url="http://0.0.0.0:5001/api/parseDocument?renderFormat=all",
# )

# docs = loader.load_and_split(
#     RecursiveCharacterTextSplitter(        
#         chunk_size=200,             
#         chunk_overlap=0,
#         separators = ["。", "！", "？"],
#         keep_separator='end',
#     ),
# )
# # 打印文档数量
# print(len(docs))
# # print(docs[0].page_content)

# # 打印所有第一页的数据出来看下，切分效果如何
# for i, item in enumerate(docs):
#     print(f"the {i} doc's content i: {item.page_content}")

In [ ]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader


# 指定 PDF 文件夹路径
pdf_directory = DOCS_DIR

# 获取文件夹中所有 PDF 文件的路径
pdf_files = [os.path.join(pdf_directory, f) for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

# 存储所有切分后的文档块
all_split_documents = []

# 对每个 PDF 文件进行处理
for pdf_file in pdf_files:
    # 使用 LLMSherpaFileLoader 加载文档
    loader = LLMSherpaFileLoader(
        file_path=pdf_file,
        new_indent_parser=True,
        apply_ocr=False,
        strategy="text",
        llmsherpa_api_url="http://0.0.0.0:5001/api/parseDocument?renderFormat=all"
    )
    
    # 使用 RecursiveCharacterTextSplitter 切分文档
    docs = loader.load_and_split(
        RecursiveCharacterTextSplitter(
            chunk_size=100,
            chunk_overlap=0,
            # separators=["。", "！", "？"],
            separators=["。"],
            keep_separator='end',
        )
    )
    
    # 将切分后的文档块添加到列表中
    all_split_documents.extend(docs)

# # 输出所有切分后的文档块
# print(all_split_documents)

In [3]:
len(all_split_documents)

8029

In [4]:
for i in range(30):
    print("##########" * 5 + f"{i+1}" + "##########" * 5)
    print(all_split_documents[i].page_content)

##################################################1##################################################
中国联通人工智能创新发展论坛在上海成功举办
发布时间：2024 年
7
月 20 日 2024 年
7
月 19 日，在中国联通合作伙伴大会期间，成功举办了人工智能创新发展论坛。
##################################################2##################################################
上海市经信委副主任张宏韬、中国联通总经理简勤、GSMA 中华区总裁斯寒出席论坛并致辞； 中国工程院院士谭建荣，加拿大工程院院士、欧洲科学院院士、香港科技大学教授郭嵩，联 通数字科技有限公司总裁、中国联通人工智能创新中心主任朱常波，中国联通人工智能科学 家兼人工智能技术总师廉士国，中国联通数字化部副总经理娄瑜发表主旨演讲。
##################################################3##################################################
上海市经信委副主任张宏韬在致辞中表示，中国联通作为中央企业，深入贯彻落实国家 “人工智能+”专项行动，在人工智能领域取得了令人瞩目的成就，中国联通人工智能创新中 心充分利用中国联通在网、算、云、数、智、端、业的融合优势，推动了人工智能创新应用 规模化发展，展现了央企在新时代的责任与担当。
##################################################4##################################################
上海市人民政府也高度重视人工智能的发 展，致力于打造开放的创新平台，吸引全球人工智能企业和人才汇聚，共同推动技术交流和 国际合作。
中国联通总经理简勤在致辞中表示，元景 2.0 不仅是中国联通人工智能技术的升级，更 是对人工智能创新发展的展望和承诺，是我们向智能时代更进一步的探索与实践。
##################################################

In [5]:
for doc in all_split_documents:
    # doc.page_content = doc.page_content.replace(" ", "").replace("\n", "")

    # 没有替换\n，因为有些表格，不应该删除换行
    doc.page_content = doc.page_content.replace(" ", "")

In [6]:
for i in range(30):
    print("##########" * 5 + f"{i+1}" + "##########" * 5)
    print(all_split_documents[i].page_content)

##################################################1##################################################
中国联通人工智能创新发展论坛在上海成功举办
发布时间：2024年
7
月20日2024年
7
月19日，在中国联通合作伙伴大会期间，成功举办了人工智能创新发展论坛。
##################################################2##################################################
上海市经信委副主任张宏韬、中国联通总经理简勤、GSMA中华区总裁斯寒出席论坛并致辞；中国工程院院士谭建荣，加拿大工程院院士、欧洲科学院院士、香港科技大学教授郭嵩，联通数字科技有限公司总裁、中国联通人工智能创新中心主任朱常波，中国联通人工智能科学家兼人工智能技术总师廉士国，中国联通数字化部副总经理娄瑜发表主旨演讲。
##################################################3##################################################
上海市经信委副主任张宏韬在致辞中表示，中国联通作为中央企业，深入贯彻落实国家“人工智能+”专项行动，在人工智能领域取得了令人瞩目的成就，中国联通人工智能创新中心充分利用中国联通在网、算、云、数、智、端、业的融合优势，推动了人工智能创新应用规模化发展，展现了央企在新时代的责任与担当。
##################################################4##################################################
上海市人民政府也高度重视人工智能的发展，致力于打造开放的创新平台，吸引全球人工智能企业和人才汇聚，共同推动技术交流和国际合作。
中国联通总经理简勤在致辞中表示，元景2.0不仅是中国联通人工智能技术的升级，更是对人工智能创新发展的展望和承诺，是我们向智能时代更进一步的探索与实践。
##################################################5##################

In [ ]:
# # 使用MinerU进行文档提取(简直无语了，巨慢，8页纸居然用了2min5s左右)
# import os

# from loguru import logger

# from magic_pdf.data.data_reader_writer import FileBasedDataWriter
# from magic_pdf.pipe.UNIPipe import UNIPipe



# try:
#     # current_script_dir = os.path.dirname(os.path.abspath(__file__))
#     # demo_name = 'demo1'
#     # pdf_path = os.path.join(current_script_dir, f'{demo_name}.pdf')

#     current_script_dir = "/root/autodl-tmp/dataset/rag/A_small"
#     demo_name = "AF01"
#     pdf_path = os.path.join(current_script_dir, f'{demo_name}.pdf')
#     pdf_bytes = open(pdf_path, 'rb').read()
#     jso_useful_key = {'_pdf_type': '', 'model_list': []}
#     local_image_dir = os.path.join(current_script_dir, 'images')
#     image_dir = str(os.path.basename(local_image_dir))
#     image_writer = FileBasedDataWriter(local_image_dir)
#     pipe = UNIPipe(pdf_bytes, jso_useful_key, image_writer)
#     # pipe.pipe_classify()
#     # pipe.pipe_analyze()
#     pipe.pipe_parse()
#     md_content = pipe.pipe_mk_markdown(image_dir, drop_mode='none')
#     with open(f'{demo_name}.md', 'w', encoding='utf-8') as f:
#         f.write(md_content)
# except Exception as e:
#     logger.exception(e)


In [ ]:
# # 使用llmsherpa
# # from llmsherpa.readers import LayoutPDFReader

# # llmsherpa_api_url = "http://127.0.0.1:5001//api/parseDocument?renderFormat=all"
# # pdf_url = "/root/autodl-tmp/dataset/rag/A_small/AF01.pdf" # also allowed is a file path e.g. /home/downloads/xyz.pdf
# # pdf_reader = LayoutPDFReader(llmsherpa_api_url)
# # doc = pdf_reader.read_pdf(pdf_url)

# from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader

# loader = LLMSherpaFileLoader(
#     file_path="/root/autodl-tmp/dataset/rag/A_small/AF01.pdf",
#     new_indent_parser=True,
#     apply_ocr=False,
#     strategy="html",
#     # llmsherpa_api_url="http://127.0.0.1:5001/api/parseDocument?renderFormat=all&useNewIndentParser=true&applyOcr=yes"
#     llmsherpa_api_url="http://0.0.0.0:5001/api/parseDocument?renderFormat=all",
# )

# docs = loader.load()




### 文本块向量化（比赛限定使用bge-large-zh-v1.5模型）

In [7]:
embeddings = HuggingFaceEmbeddings(model_name=EMB_MODEL, show_progress=True)
vectordb = FAISS.from_documents(   
    documents=all_split_documents,
    embedding=embeddings,
)

vectordb.save_local(PERSIST_DIR)

/tmp/ipykernel_7939/2932526079.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMB_MODEL, show_progress=True)
Batches: 100%|██████████| 251/251 [00:54<00:00,  4.64it/s]


### 混合检索器

#### bm25 
- k1 较高的 k1 值意味着词频对评分的影响更大。
- b  当 b=1 时，文档长度的影响最大；当b = 0 时，文档长度不影响评分。
- langchain 默认切分英文split()，中文需要jieba分词

In [8]:
import jieba
dense_retriever = vectordb.as_retriever(search_kwargs={"k": 5})
bm25_retriever = BM25Retriever.from_documents(
    all_split_documents, 
    k=5, 
    bm25_params={"k1": 1.5, "b": 0.75}, 
    preprocess_func=jieba.lcut
)
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, dense_retriever], weights=[0.5, 0.5])

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.639 seconds.
Prefix dict has been built successfully.


### 文本召回和重排

In [9]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

def rerank(questions, retriever, top_n=5, cut_len=384):
    rerank_model = HuggingFaceCrossEncoder(model_name=RERANK_MODEL)
    compressor = CrossEncoderReranker(model=rerank_model, top_n=top_n)
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=retriever
    )
    rerank_answers = []
    for question in tqdm(questions):
        relevant_docs = compression_retriever.invoke(question)
        answer=''
        for rd in relevant_docs:
            answer += rd.page_content
        rerank_answers.append(answer[:cut_len])
    return rerank_answers

questions = list(query['question'].values)
rerank_answers = rerank(questions, ensemble_retriever)
print(rerank_answers[0])

100%|██████████| 100/100 [00:11<00:00,  9.05it/s]



我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字科技领军企业转变，实现了四个维度的转型升级：一是联接规模和联接结构升维，从过去的连接人为主拓展到连接人机物，大力发展物联网和工业互联网；二是核心功能升维，从以基础连接为主发展到大联接、大计算、大数据、大应用、大安全五大主责主业；三是服务和赋能水平升维，以5G、云计算、大数据、人工智能、区块链为代表的新一代信息技术和实体经济的结合，服务数字政府、数字社会、数字经济的能力不断增强；四是发展理念升维，我们以传统的市场驱动为主转变为市场驱动和创新驱动相结合的发展模式，尤其是加大了科技创新及人才方面的投入力度，创新发展的动能得到了空前的释放。2022年研发费用同比增长43%，科技创新人员占比达到30%，授权专利数达到1,666件，自主研发产品收入同比增长超过70%。加大核心技


### 提交

In [10]:
def emb(answers, emb_batch_size = 4):
    model = SentenceTransformer(EMB_MODEL, trust_remote_code=True).half()
    all_sentence_embeddings = []
    for i in tqdm(range(0, len(answers), emb_batch_size), desc="embedding sentences"):
        batch_sentences = answers[i:i+emb_batch_size]
        sentence_embeddings = model.encode(batch_sentences, normalize_embeddings=True)
        all_sentence_embeddings.append(sentence_embeddings)
    all_sentence_embeddings = np.concatenate(all_sentence_embeddings, axis=0)
    print('emb_model max_seq_length: ', model.max_seq_length)
    print('emb_model embeddings_shape: ', all_sentence_embeddings.shape[-1])
    del model
    gc.collect()
    torch.cuda.empty_cache()
    return all_sentence_embeddings

all_sentence_embeddings = emb(rerank_answers)
sub['answer'] = rerank_answers
sub['embedding']= [','.join([str(a) for a in all_sentence_embeddings[i]]) for i in range(len(all_sentence_embeddings))]
sub.to_csv(SUB_DIR, index=None)
sub.head()

embedding sentences: 100%|██████████| 25/25 [00:00<00:00, 33.87it/s]


emb_model max_seq_length:  512
emb_model embeddings_shape:  1024


,ques_id,question,answer,embedding
0,1,根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？,\n我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向...,"-0.02783,-0.00862,-0.014114,0.0008893,0.01552,..."
1,2,告诉我2022年联通产业互联网收入的同比增长速度。,年内，产业互联网业务收入同比增长30.0%，达到人民币427亿元，占整体主营业务收入比例提高...,"0.0001297,0.0008335,-0.01787,-0.004925,0.0144,..."
2,3,根据2022年度报告，中国联通的企业定位是什么？,2022年是公司新战略落地实施的关键之年。中国联通将持续全面贯彻新发展理念，服务构建新发展格...,"-0.02388,0.002645,-0.03522,0.0191,0.02032,0.06..."
3,4,2022年联通在“大联接”和“大数据”业务上取得了什么成果？,"2022年研发费用同比增长43%，科技创新人员占比达到30%，授权专利数达到1,666件，自...","-0.03558,-0.01369,-0.022,0.007114,0.01347,0.03..."
4,5,2022年上半年，联通在精品网络建设上有什么成果？,流量释放成效显著，手机上网总流量增长13.4%，手机用户月户均数据流量达到约13.3GB。公...,"-0.004036,-0.02444,-0.05856,0.001368,0.02396,0..."


In [11]:
for i in range(10,20):
    print("##########"* 5, f"{i+1}", "##########" * 5)
    print(rerank_answers[i])


################################################## 11 ##################################################
现将2022年度的履职情况汇报如下：一、董事会审计委员会履职情况（一）审计委员会基本情况报告期内，公司董事会审计委员会由6名董事组成，包括吴晓根董事、王军辉董事、顾佳丹董事、高云虎董事、鲍朔望董事、童国华董事。现将2019年度的履职情况汇报如下：
一、董事会审计委员会履职情况
（一）审计委员会基本情况
报告期内，公司董事会审计委员会由5名董事组成，包括吴晓根董事、尹兆君董事、冯士栋董事、陈建新董事和李福申董事。现将2020年度的履职情况汇报如下：
一、董事会审计委员会履职情况
（一）审计委员会基本情况
报告期内，公司董事会审计委员会由5名董事组成，包括吴晓根董事、尹兆君董事、冯士栋董事、陈建新董事和李福申董事。报告期内，审计委员会召开情况具体如下：
|会议届次|召开时间|会议地点及方式（会议表决通过，并提交董事会）
|---|---|---
|第七届董事会审
################################################## 12 ##################################################

第二节公司基本情况
2.1公司简介
公司股票简况
|股票种类|股票上市交易所|股票简称|股票代码
|---|---|---|---
|A股|上海证券交易所|中国联通|600050
|联系人和联系方式|董事会秘书|证券事务代表|
|姓名|李玉焯|雷晓旭|
|电话|010-66259179|010-66259179|
|办公地址|北京市西城区金融大街21号|北京市西城区金融大街21号|
|电子信箱|dongmi@chinaunicom.cn|ir@chinaunicom.cn|
|2.2主要财务数据单位：元币种：人民币
||本报告期末|上年度末本报告期末比上年度末增减(%)|
|总资产|605,441,750,050|593,284,479,854|2.0
|归属于上市公司股东的净资产|153,865,868,606|149,217,148,845|3.1
|
####################

### 后续可能提分点
- 引入LLM
   * LLM 递归判断/抽取
   * rag-fusion 查询改写
   * 构建知识图谱



### 注意：
- 在分块、重排等过程中可以使用公开库和模型，但禁止使用LLM直接生成最终答案。
- 禁止使用LLM继续调整精排得到的文本块，如压缩文本块长度；
- 禁止使用LLM直接从文档获取问题答案。